In [1]:
!nvidia-smi

Fri Jan 31 11:46:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.58.02              Driver Version: 555.58.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        Off |   00000000:49:00.0 Off |                  N/A |
| 32%   31C    P8             25W /  420W |   16987MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

stable models

| models              | versions| Date | features | pretrained |
| --------            | ------- | -----| ------- | ------- |
| stable diffusion 1  | 1.1-1.5 | 6/22 | text: ViT-L/14 CLIP model<br>resolution: 77toks, 512X512<br>parameters: ~860M<br>quality: general usage, flat-looking | runwayml/stable-diffusion-v1-5 |
| stable diffusion 2  | 2.0 2.1 | 11/22| text: ViT-H/14 CLIP model<br>resolution: 77toks, 768X768<br>parameters: ~860M<br>quality: general usage, flat-looking | stabilityai/stable-diffusion-2-1 |
| stable diffusion XL | 1.0     | 6/23 | text: OpenCLIP-ViT model<br>resolution: 77toks, 1024X1024<br>parameters: ~3.5B<br>quality: general usage, high-res | stable-diffusion-xl-base-1.0 |
| stable diffusion XL turbo |   | 11/23| text: OpenCLIP-ViT model<br>resolution: 77toks, 512X512<br>parameters: ~3.5B<br>quality: general usage, high-res | stabilityai/sdxl-turbo |
| stable cascade      |         | 2/24 | Würstchen architecture - research only| stabilityai/stable-cascade |
| stable diffusion 3  | 3.0 3.5 | 3/24 |  | stabilityai/stable-diffusion-3-medium-diffusers |
| kandinsky           |         |      |  |  |

ref:
- Würstchen architecture: https://openreview.net/pdf?id=gU58d5QeGv
- Comparing Stable Diffusion Models: https://medium.com/@promptingpixels/comparing-stable-diffusion-models-2c1dc9919ab7
- Survey Conditional Image Synthesis with Diffusion Models: https://arxiv.org/pdf/2409.19365
- controlnet: https://arxiv.org/pdf/2302.05543

In [4]:
from diffusers import AutoPipelineForImage2Image
import torch
pipe = AutoPipelineForImage2Image.from_pretrained("runwayml/stable-diffusion-v1-5", use_safetensors=True, torch_dtype=torch.float16, variant="fp16",)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
count = sum([l.numel() for l in pipe.unet.parameters()])
count

859520964

In [ ]:
prev_t = max(1, t.item() - (1000//num_inference_steps)) # t-1
alpha_t = pipe.scheduler.alphas_cumprod[t.item()]
alpha_t_prev = pipe.scheduler.alphas_cumprod[prev_t]
predicted_x0 = (latents - (1-alpha_t).sqrt()*noise_pred) / alpha_t.sqrt()
direction_pointing_to_xt = (1-alpha_t_prev).sqrt()*noise_pred
latents = alpha_t_prev.sqrt()*predicted_x0 + direction_pointing_to_xt